In [1]:
from model import CRNN
from Dataset.CropDataset import CropDataset
from transforms import get_transforms
import torch
import string
import matplotlib.pyplot as plt

alphabet = " "
alphabet += string.ascii_uppercase
alphabet += "".join([str(i) for i in range(10)])

MODEL_PARAMS = {
    'image_height': 32,
    'number_input_channels': 3,
    'number_class_symbols': len(alphabet),
    'rnn_size': 12
}

model_data = torch.load('logs/checkpoints/best_full.pth')
model = CRNN(**MODEL_PARAMS)
model.load_state_dict(model_data['model_state_dict'])

device=torch.device('cpu:0')

dataset = CropDataset(transforms=get_transforms(device), cached=False)

In [2]:
len(dataset.Train)

2920

In [3]:
ds = dataset.Train[10]
ds[0].shape

(3, 32, 80)

In [4]:
result = model(torch.Tensor([ds[0]]))
result.shape, len(alphabet)

(torch.Size([22, 1, 37]), 37)

In [5]:
values, indexes = result.max(0)

print(values)
print(indexes)

tensor([[-0.4121, -3.9205, -4.2587, -3.7664, -3.2591, -3.4394, -3.8954, -4.2779,
         -3.6079, -4.2631, -3.9689, -3.7693, -3.4062, -3.9924, -4.5980, -3.9499,
         -3.5784, -4.0597, -3.6008, -3.8323, -3.8660, -4.0715, -3.5061, -4.1871,
         -3.5006, -4.0156, -4.2781, -3.9142, -3.2327, -3.5920, -3.5141, -3.0901,
         -4.3384, -3.6198, -3.7352, -4.0412, -4.1087]], grad_fn=<MaxBackward0>)
tensor([[12, 21, 21, 21,  0,  0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
         21, 21, 21,  0, 21,  0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
         21]])


In [6]:
indexes = indexes.numpy()[0]
indexes

array([12, 21, 21, 21,  0,  0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21, 21,  0, 21,  0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 21, 21])

In [7]:
from converter import strLabelConverter

conv = strLabelConverter(alphabet)

res = "".join([alphabet[i] for i in indexes])
res

'LUUU  UUUUUUUUUUUUUUU U UUUUUUUUUUUUU'